In [1]:
!nvidia-smi

Fri Mar 14 15:25:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.04             Driver Version: 570.124.04     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 4000 Ada Gene...    Off |   00000000:01:00.0  On |                  Off |
| 30%   31C    P8             14W /  130W |    1069MiB /  20475MiB |     35%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

In [3]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq,
)

/users/eleves-a/2024/christelle.clervilsson/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib64/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

bnb_config = {
    "load_in_4bit": True,  
    "bnb_4bit_compute_dtype": torch.float16,  # Ensure computation is done in float16
    "torch_dtype": torch.float16,  # Use float16 for computation
    "device_map": "auto",  
    "llm_int8_enable_fp32_cpu_offload": True, 
}
model_name_I= "Qwen/Qwen2.5-0.5B-Instruct"
model_name = "Qwen/Qwen2.5-0.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
config = LoraConfig(r=32,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc_in", "fc_out", "wte"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


In [43]:
prompt=" Résumez cet article \n<article>: Deux images, et un sujet central : l’Ukraine. La journée du jeudi 13 mars a été marquée par un dialogue public à distance entre Vladimir Poutine et Donald Trump. Accueillant à Moscou son homologue biélorusse, Alexandre Loukachenko, le président russe a acquiescé du bout des lèvres, et sous conditions, à la proposition de cessez-le-feu de trente jours, avancée par la Maison Blanche et validée par Kiev. Pendant ce temps, à Washington, Donald Trump recevait le secrétaire général de l’OTAN, Mark Rutte. Le rapprochement américano-russe et l’acception conditionnée de la solidarité entre alliés, prisée par le milliardaire, inquiètent fortement les membres européens de l’Alliance atlantique. Au terme de cette journée, ni la paix en Ukraine ni l’avenir de l’OTAN ne se dessinent plus clairement.\n<résumé>:"
print(prompt)

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.do_sample = True

 Résumez cet article 
<article>: Deux images, et un sujet central : l’Ukraine. La journée du jeudi 13 mars a été marquée par un dialogue public à distance entre Vladimir Poutine et Donald Trump. Accueillant à Moscou son homologue biélorusse, Alexandre Loukachenko, le président russe a acquiescé du bout des lèvres, et sous conditions, à la proposition de cessez-le-feu de trente jours, avancée par la Maison Blanche et validée par Kiev. Pendant ce temps, à Washington, Donald Trump recevait le secrétaire général de l’OTAN, Mark Rutte. Le rapprochement américano-russe et l’acception conditionnée de la solidarité entre alliés, prisée par le milliardaire, inquiètent fortement les membres européens de l’Alliance atlantique. Au terme de cette journée, ni la paix en Ukraine ni l’avenir de l’OTAN ne se dessinent plus clairement.
<résumé>:


In [44]:
%%time

device ="cuda" if torch.cuda.is_available() else "cpu"
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

 Résumez cet article 
<article>: Deux images, et un sujet central : l’Ukraine. La journée du jeudi 13 mars a été marquée par un dialogue public à distance entre Vladimir Poutine et Donald Trump. Accueillant à Moscou son homologue biélorusse, Alexandre Loukachenko, le président russe a acquiescé du bout des lèvres, et sous conditions, à la proposition de cessez-le-feu de trente jours, avancée par la Maison Blanche et validée par Kiev. Pendant ce temps, à Washington, Donald Trump recevait le secrétaire général de l’OTAN, Mark Rutte. Le rapprochement américano-russe et l’acception conditionnée de la solidarité entre alliés, prisée par le milliardaire, inquiètent fortement les membres européens de l’Alliance atlantique. Au terme de cette journée, ni la paix en Ukraine ni l’avenir de l’OTAN ne se dessinent plus clairement.
<résumé>: C'est une histoire d'histoire politique qui démarre avec l'entrée au pouvoir de Vladimir Poutine, le 24 janvier 2015, dans l'émeute de l'URSS, et qui s'acheit a

In [ ]:
ds = pd.read_csv("summary_dataset.csv")
train_ds, temp_ds = ds.train_test_split(test_size=0.2).values()  # 80% train, 20% temp
valid_ds, test_ds = temp_ds.train_test_split(test_size=0.5).values()  # Split temp into 50% validation, 50% test


# Display the number of examples in each split
print(f"Training Set: {len(train_ds)} examples")
print(f"Validation Set: {len(valid_ds)} examples")
print(f"Test Set: {len(test_ds)} examples")

Training Set: 4000 examples
Validation Set: 500 examples
Test Set: 500 examples


In [12]:
def generate_summary(example):
    prompt= example['text']
    encoding = tokenizer( " Résumez cet article \n<article>: "+prompt+ "\n<résumé>:", return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    summary =tokenizer.decode(outputs[0], skip_special_tokens=True)
    example['previous-summary-no-shot'] = summary
    return example

# Iterate over the dataset and generate summaries
test_ds=test_ds.map(generate_summary)
# Save the modified dataset with the new column
test_ds.to_csv("orangesum_with_summary_base_model_noshot.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 19.77ba/s]


3212241

In [ ]:
def generate_summaryO(example):
    prompt= example['text']
    encoding = tokenizer( "Voici l'exemples d' un article et son résumé :\n Article : Un réseau d'escrocs qui revendait des voitures de luxe achetées frauduleusement a été démantelé. L'opération a mobilisé 90 gendarmes en Gironde, dans le Var et les Pyrénées-Orientales. Dix suspects ont été arrêtés, avec un préjudice total de 2 millions d'euros.\n Résumé : Dix personnes arrêtées après une enquête sur une escroquerie aux voitures de luxe, causant un préjudice de 2 millions d’euros.\nMaintenant résumez cet article \n<article>: "+prompt+ "\n<résumé>:", return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )

    summary =tokenizer.decode(outputs[0], skip_special_tokens=True)
    example['previous-summary-one-shot'] = summary
    return example
test_ds=test_ds.map(generate_summaryO)
test_ds.to_csv("orangesum_with_summary_base_model_oneshot.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 11.87ba/s]


5149558

In [ ]:
def generate_summaryF(example):
    prompt= example['text']
    encoding = tokenizer( "Voici des exemples d'articles et leurs résumés : \nArticle : Un réseau d'escrocs qui revendait des voitures de luxe achetées frauduleusement a été démantelé. L'opération a mobilisé 90 gendarmes en Gironde, dans le Var et les Pyrénées-Orientales. Dix suspects ont été arrêtés, avec un préjudice total de 2 millions d'euros. \n Résumé : Dix personnes arrêtées après une enquête sur une escroquerie aux voitures de luxe, causant un préjudice de 2 millions d’euros. \n Article : La saison 7 de Danse avec les stars s'est terminée, mais la saison 8 pourrait voir un changement majeur. Marie-Claude Pietragalla, juge depuis 2012, a indiqué qu'elle pourrait ne pas revenir en raison d'un emploi du temps chargé. \n Résumé :  Marie-Claude Pietragalla pourrait quitter Danse avec les stars avant la saison 8. \n Maintenant, résumez cet article \n<article>: "+prompt+ "\n<résumé>:", return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )

    summary =tokenizer.decode(outputs[0], skip_special_tokens=True)
    example['previous-summary-few-shot'] = summary
    return example
test_ds=test_ds.map(generate_summaryF)
test_ds.to_csv("orangesum_with_summary_base_model_fewshot.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  8.20ba/s]


7236904

In [ ]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = f"Résumez cet article,\n<article>: {data_point['text']}\n<résumé>:"
    response = f" {data_point['generated-summary']}"  

    full_prompt_with_response = full_prompt + response + tokenizer.eos_token  # Append response + EOS token
    tokenized_full_prompt = tokenizer(
        full_prompt_with_response,
        padding="max_length",
        truncation=True,
        return_tensors='pt',
        max_length=512
    )
    labels = tokenized_full_prompt.input_ids.clone()
    end_prompt_idx = len(tokenizer(full_prompt)['input_ids']) 
    labels[:, :end_prompt_idx] = -100
    return {
        'input_ids': tokenized_full_prompt.input_ids.flatten(),
        'labels': labels.flatten(),
        'attention_mask': tokenized_full_prompt.attention_mask.flatten(),
    }
train_dataset = train_ds.shuffle(seed=42).map(generate_and_tokenize_prompt)
val_dataset=valid_ds.shuffle(seed=42).map(generate_and_tokenize_prompt)

Map: 100%|██████████| 500/500 [00:00<00:00, 559.97 examples/s]


In [47]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import torch
data_collator = DataCollatorForSeq2Seq(tokenizer, model)

training_args = TrainingArguments(
  output_dir='./results',
    evaluation_strategy='steps',
    eval_steps=200,
    logging_dir='./logs',
    logging_steps=200,
    save_steps=1000,
    save_total_limit=3,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    adam_epsilon=1e-8,
    adam_beta1=0.9,
    adam_beta2=0.999,
    gradient_accumulation_steps=2,
    warmup_steps=500,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
)
torch.cuda.empty_cache()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,

)
model.config.use_cache = False
trainer.train()


/users/eleves-a/2024/christelle.clervilsson/.local/lib/python3.9/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_2576282/4172712129.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names 

Step,Training Loss,Validation Loss
200,2.967800,nan
400,2.301100,nan
600,2.131900,nan


TrainOutput(global_step=750, training_loss=2.364438761393229, metrics={'train_runtime': 576.1801, 'train_samples_per_second': 20.827, 'train_steps_per_second': 1.302, 'total_flos': 1.3302272360448e+16, 'train_loss': 2.364438761393229, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./saved_model23")
tokenizer.save_pretrained("./saved_model23_tokenizer")

('./saved_model23_tokenizer-Instruct/tokenizer_config.json',
 './saved_model23_tokenizer-Instruct/special_tokens_map.json',
 './saved_model23_tokenizer-Instruct/vocab.json',
 './saved_model23_tokenizer-Instruct/merges.txt',
 './saved_model23_tokenizer-Instruct/added_tokens.json',
 './saved_model23_tokenizer-Instruct/tokenizer.json')

In [37]:
prompt=" Résumez cet article \n<article>: Deux images, et un sujet central : l’Ukraine. La journée du jeudi 13 mars a été marquée par un dialogue public à distance entre Vladimir Poutine et Donald Trump. Accueillant à Moscou son homologue biélorusse, Alexandre Loukachenko, le président russe a acquiescé du bout des lèvres, et sous conditions, à la proposition de cessez-le-feu de trente jours, avancée par la Maison Blanche et validée par Kiev. Pendant ce temps, à Washington, Donald Trump recevait le secrétaire général de l’OTAN, Mark Rutte. Le rapprochement américano-russe et l’acception conditionnée de la solidarité entre alliés, prisée par le milliardaire, inquiètent fortement les membres européens de l’Alliance atlantique. Au terme de cette journée, ni la paix en Ukraine ni l’avenir de l’OTAN ne se dessinent plus clairement.\n<résumé>:"

In [38]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

 Résumez cet article 
<article>: Deux images, et un sujet central : l’Ukraine. La journée du jeudi 13 mars a été marquée par un dialogue public à distance entre Vladimir Poutine et Donald Trump. Accueillant à Moscou son homologue biélorusse, Alexandre Loukachenko, le président russe a acquiescé du bout des lèvres, et sous conditions, à la proposition de cessez-le-feu de trente jours, avancée par la Maison Blanche et validée par Kiev. Pendant ce temps, à Washington, Donald Trump recevait le secrétaire général de l’OTAN, Mark Rutte. Le rapprochement américano-russe et l’acception conditionnée de la solidarité entre alliés, prisée par le milliardaire, inquiètent fortement les membres européens de l’Alliance atlantique. Au terme de cette journée, ni la paix en Ukraine ni l’avenir de l’OTAN ne se dessinent plus clairement.
<résumé>: Le 13 mars 2021, le président russe Vladimir Poutine a déclaré à Washington que la Ukraine serait « déployée à la frontière de l’Europe ». Le Kremlin a présenté

In [ ]:
def generate_summaryFT(example):
    prompt= example['text']
    encoding = tokenizer( " Résumez cet article \n<article>: "+prompt+ "\n<résumé>:", return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    summary =tokenizer.decode(outputs[0], skip_special_tokens=True)
    example['current-summary'] = summary
    return example

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_ds=test_ds.map(generate_summaryFT)
test_ds.to_csv("orangesum_with_summary_afterfinetuning-Instruct.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 19.20ba/s]


3256199